# Configuración para la arquitectura del proyecto

## Collection de Amazon Rekognition

Primero se crea una coleccion en amazon rekognition para almacenar la cara de las personas que serán identificadas posteriormente, amazon solo comparara las imagenes entrantes al sistema contra aquellas almacenadas en la collection 

In [1]:
import boto3

# La función create_collection, crea la collection 
# recibe el nombre de la collection que se desea crear 

def create_collection(collection_id):

    #Se hace el llamdo al cliente de amazon rekognition 
    client=boto3.client('rekognition')

    #Creación de la collection
    response=client.create_collection(CollectionId=collection_id)
    print('Collection ' + collection_id + ' creada')
    
def main():
    collection_id='collection-rekognition'
    create_collection(collection_id)

if __name__ == "__main__":
    main()  

Creating collection:collection-rekognition
Collection ARN: aws:rekognition:us-east-1:853702706419:collection/collection-rekognition
Status code: 200
Done...


Se añaden las caras a la collection, se da el path del local o el bucket donde se encuentran las imagenes que se desean agregar a la collection, principalmente estas imagenes tendran un faceId, ImageID y externalImageID, en la collection

In [37]:
import boto3
import os

#En este caso agregamos las imagenes desde el path del local 
path =r'C:\Users\user\Desktop\collection'
fpath = os.listdir(path)

diccionario = {}


def add_faces_to_collection(target_file, photo, collection_id, region):
            client = boto3.client('rekognition', region_name=region)

            imageTarget = open(target_file, 'rb')

            response = client.index_faces(CollectionId=collection_id,
                                        Image={'Bytes': imageTarget.read()},
                                        ExternalImageId=photo,
                                        MaxFaces=1,
                                        QualityFilter="AUTO")
        
            diccionario[response['FaceRecords'][0]['Face']['FaceId']] = images
            print(response['FaceRecords'][0]['Face']['FaceId'])

for folder in fpath:
    path_folders =  path +'\\' + folder
    fpath_folders = os.listdir(path_folders)
    for images in fpath_folders:
        image = r''+path + '\\'+ folder + '\\'+ images
        collection_id = 'collection-rekognition'
        photo_name = folder
        region = "us-east-1"
        print(images)
        
                                        
        indexed_faces_count = add_faces_to_collection(image, photo_name, collection_id, region)
            



        

Emanuel.jpg
645ad99f-fed7-4e4b-bfda-b443e3ef0bcb
Emanuel2.jpg
384ef251-39fd-4dae-ae94-f5379711a09b
Emanuel3.jpg
e9039887-0158-45f8-bb89-030ce49c3847
Ibra.jpg
6011be88-c031-4eb4-bb12-6f8ca4712f17
Ibra2.jpg
5f9b809c-ea30-478b-9153-09905b081dcc
ibra3.jpg
6d3043cd-2ea5-4fff-8516-cfcc432196b5
Messi.jpg
96b557aa-bec4-418a-a59a-d89f42b4408e
Messi2.jpg
2f7e32a6-8f2d-4220-97f3-18e940a074a9
Messi3.jpg
4c4f9284-51c9-48c8-8baf-75197c5c31de
Ronaldo.jpg
46d9fdab-4be1-4e42-af3f-ea7704f5d7e7
Ronaldo2.jpg
5aeef840-b4cb-450a-8a00-2db999a511e4
Ronaldo3.jpg
f766139f-cf17-473d-92bd-dd931028a6ee


Describir collection

In [1]:
import boto3
from botocore.exceptions import ClientError

def describe_collection(collection_id):

    print('Attempting to describe collection ' + collection_id)
    client=boto3.client('rekognition')

    try:
        response=client.describe_collection(CollectionId=collection_id)
        print("Collection Arn: "  + response['CollectionARN'])
        print("Face Count: "  + str(response['FaceCount']))
        print("Face Model Version: "  + response['FaceModelVersion'])
        print("Timestamp: "  + str(response['CreationTimestamp']))

        
    except ClientError as e:
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            print ('The collection ' + collection_id + ' was not found ')
        else:
            print ('Error other than Not Found occurred: ' + e.response['Error']['Message'])
    print('Done...')


def main():
    collection_id='collection-rekognition'
    describe_collection(collection_id)

if __name__ == "__main__":
    main()

Attempting to describe collection collection-rekognition
Collection Arn: arn:aws:rekognition:us-east-1:853702706419:collection/collection-rekognition
Face Count: 12
Face Model Version: 6.0
Timestamp: 2022-08-10 13:47:28.168000-05:00
Done...


Creamos tabla de dynamoDB

In [ ]:
from pprint import pprint
import boto3

# Create DynamoDB database with image URL and face data, face ID

def create_dynamodb_table(table_name, region):
    dynamodb = boto3.client("dynamodb", region_name=region)

    table = dynamodb.create_table(
        TableName=table_name,   
        KeySchema=[{
                'AttributeName': 'FaceID', 'KeyType': 'HASH'  # Partition key  
                },],        
            AttributeDefinitions=[
            {
                'AttributeName': 'FaceID', 'AttributeType': 'S'  }, ],        
                ProvisionedThroughput={
            'ReadCapacityUnits': 10, 'WriteCapacityUnits': 10  }
    )
    pprint(table)
    return table

region = "us-east-1"
database_name = 'dataset-collection-images'
dynamodb_table = create_dynamodb_table(database_name, region)
print("Table status:", dynamodb_table)

Subimos imagenes de la Collection a S3

In [5]:
import boto3
import logging
from botocore.exceptions import ClientError


for folder in fpath:
    path_folders =  path +'\\' + folder
    fpath_folders = os.listdir(path_folders)
    for images in fpath_folders:
        file_name = r''+path + '\\'+ folder + '\\'+ images
        # store local file in S3 bucket
        bucket = "prueba-bucket-machine"
        key_name = folder+'/'+ images
        region = "us-east-1"
        s3 = boto3.client('s3', region_name=region)
        # Upload the file
        try:
            response = s3.upload_file(file_name, bucket, key_name)
            print("File upload successful!")
        except ClientError as e:
            logging.error(e)

File upload successful!
File upload successful!
File upload successful!
File upload successful!
File upload successful!
File upload successful!
File upload successful!
File upload successful!
File upload successful!
File upload successful!
File upload successful!
File upload successful!


Listado de caras

In [39]:
import boto3

lista = []


def list_faces_in_collection(collection_id):


    maxResults=1
    faces_count=0
    tokens=True

    client=boto3.client('rekognition')
    response=client.list_faces(CollectionId=collection_id,
                               MaxResults=maxResults)

    print('Faces in collection ' + collection_id)

 
    while tokens:

        list1 = []
        faces=response['Faces']

        for face in faces:
            list1.append(face['FaceId'])
            list1.append(face['ImageId'])
            list1.append(face['ExternalImageId'])
            lista.append(list1)
            list1.clear

            faces_count+=1
        if 'NextToken' in response:
            nextToken=response['NextToken']
            response=client.list_faces(CollectionId=collection_id,
                                       NextToken=nextToken,MaxResults=maxResults)
        else:
            tokens=False
    print(lista)
    return faces_count   
def main():

    collection_id='collection-rekognition'

    faces_count=list_faces_in_collection(collection_id)
    print("faces count: " + str(faces_count))
if __name__ == "__main__":
    main()

Faces in collection collection-rekognition
[['2f7e32a6-8f2d-4220-97f3-18e940a074a9', '29e23352-48a3-3199-893e-cbbfb99ebf83', 'Messi'], ['384ef251-39fd-4dae-ae94-f5379711a09b', 'd1fee435-88f4-3319-b027-812dd107c0a4', 'Emanuel'], ['46d9fdab-4be1-4e42-af3f-ea7704f5d7e7', 'e5f3c369-42a7-32ac-a67a-8e093d1219ec', 'Ronaldo'], ['4c4f9284-51c9-48c8-8baf-75197c5c31de', 'dd3259ec-31fc-3c92-afe0-feca9ebd5728', 'Messi'], ['5aeef840-b4cb-450a-8a00-2db999a511e4', 'bb0b58c7-eac4-3c2d-a3ee-74fcc10650e1', 'Ronaldo'], ['5f9b809c-ea30-478b-9153-09905b081dcc', '6dcb7e95-e777-37fc-a614-d8769061417e', 'Ibrahimovic'], ['6011be88-c031-4eb4-bb12-6f8ca4712f17', '4c5e0f65-4281-3cae-a707-c5bd260bad78', 'Ibrahimovic'], ['645ad99f-fed7-4e4b-bfda-b443e3ef0bcb', '91a0a87b-bcef-394c-89ed-57eb0e6d0577', 'Emanuel'], ['6d3043cd-2ea5-4fff-8516-cfcc432196b5', 'a8ee50f6-4b74-3f43-9c2a-0411747efa59', 'Ibrahimovic'], ['96b557aa-bec4-418a-a59a-d89f42b4408e', '538cb8f2-1b34-38c1-b078-0bddc8ca918c', 'Messi'], ['e9039887-0158-45f8

In [18]:
client = boto3.client('s3')

bucketName = "prueba-bucket-machine"

keys = []

response = client.list_objects(
            Bucket=bucketName,
            MaxKeys=123,
        )

for key in response['Contents']:
    keys.append(key['Key'])
print(keys)

['Emanuel/Emanuel.jpg', 'Emanuel/Emanuel2.jpg', 'Emanuel/Emanuel3.jpg', 'Ibrahimovic/Ibra.jpg', 'Ibrahimovic/Ibra2.jpg', 'Ibrahimovic/ibra3.jpg', 'Messi/Messi.jpg', 'Messi/Messi2.jpg', 'Messi/Messi3.jpg', 'Ronaldo/Ronaldo.jpg', 'Ronaldo/Ronaldo2.jpg', 'Ronaldo/Ronaldo3.jpg']


Trasladar de s3 a dynamoDB

In [42]:
import boto3
from pprint import pprint
from decimal import Decimal
import json

bucket = "prueba-bucket-machine"
for item in lista:
    for key in keys:
        list = key.split('/')
        if list[0]==item[2]:
            # The local file that was stored in S3 bucket
            file_name = '/'+ diccionario[item[0]]
            key_name = key
            region = "us-east-1"
            # Get URL of file
            file_url = "https://s3.amazonaws.com/{}/{}".format(bucket, key_name)
            print(file_url)

            # upload face-id, face info, and image url
            def AddDBEntry(name, file_url, face_id, image_id,status):
                dynamodb = boto3.resource('dynamodb', region_name=region)
                table = dynamodb.Table('dataset-collection-images')
                response = table.put_item(
                Item={
                        'ExternalImageID': name,
                        'ImageURL': file_url,
                        'FaceID': face_id,
                        'ImageID': image_id, 
                        'status': status
                }
                )
                return response

            # Mock values for face ID, image ID, and confidence - replace them with actual values from your collection results
            try:
                dynamodb_resp = AddDBEntry(item[2], file_url, item[0],  
                item[1],False)
            except:
                print("El usuario ya se encuentra en la tabla")
            print("Database entry successful.")
            pprint(dynamodb_resp, sort_dicts=False)

https://s3.amazonaws.com/prueba-bucket-machine/Messi/Messi.jpg
Database entry successful.
{'ResponseMetadata': {'RequestId': 'GLAESRSB0BDD3V579DNFH3QPNFVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'HTTPStatusCode': 200,
                      'HTTPHeaders': {'server': 'Server',
                                      'date': 'Wed, 10 Aug 2022 20:53:01 GMT',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'content-length': '2',
                                      'connection': 'keep-alive',
                                      'x-amzn-requestid': 'GLAESRSB0BDD3V579DNFH3QPNFVV4KQNSO5AEMVJF66Q9ASUAAJG',
                                      'x-amz-crc32': '2745614147'},
                      'RetryAttempts': 0}}
https://s3.amazonaws.com/prueba-bucket-machine/Messi/Messi2.jpg
Database entry successful.
{'ResponseMetadata': {'RequestId': '5CU1TKFL194CT5O59HR6V354LFVV4KQNSO5AEMVJF66Q9ASUAAJG',
                   

In [40]:
print(diccionario)

{'645ad99f-fed7-4e4b-bfda-b443e3ef0bcb': 'Emanuel.jpg', '384ef251-39fd-4dae-ae94-f5379711a09b': 'Emanuel2.jpg', 'e9039887-0158-45f8-bb89-030ce49c3847': 'Emanuel3.jpg', '6011be88-c031-4eb4-bb12-6f8ca4712f17': 'Ibra.jpg', '5f9b809c-ea30-478b-9153-09905b081dcc': 'Ibra2.jpg', '6d3043cd-2ea5-4fff-8516-cfcc432196b5': 'ibra3.jpg', '96b557aa-bec4-418a-a59a-d89f42b4408e': 'Messi.jpg', '2f7e32a6-8f2d-4220-97f3-18e940a074a9': 'Messi2.jpg', '4c4f9284-51c9-48c8-8baf-75197c5c31de': 'Messi3.jpg', '46d9fdab-4be1-4e42-af3f-ea7704f5d7e7': 'Ronaldo.jpg', '5aeef840-b4cb-450a-8a00-2db999a511e4': 'Ronaldo2.jpg', 'f766139f-cf17-473d-92bd-dd931028a6ee': 'Ronaldo3.jpg'}
